# 필요 라이브러리 import

In [78]:
import selenium
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from openpyxl import Workbook
from selenium.webdriver import ActionChains

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pyautogui
import mouseinfo

import time
import datetime
import requests
import random


import os
import pandas as pd
from tqdm import tqdm

# 크롬 드라이버 셋팅 

In [79]:
url = "https://www.diningcode.com/list.dc?query="
# 협업을 위해 각자 "00동 음식점, 00동 카페, 00동 술집" 알아서 넣는걸로 하자!
# csv 이름도 바꿔야함 ex) 군자동 카페 => gunja_cafe
location="화양동 카페"

#플랫폼은 다이닝코드 고정이라서 건들 필요 없음
global platform
platform=1


"""
음식점 : 0
카페 : 1
술집 : 2

크롤링 카테고리에 맞게 변수 바꿔야함
"""
global category
category=1

In [80]:
options = Options()
# Chrome의 binary path 지정
options.binary_location = (
    "/Users/powerjsv/Desktop/Google Chrome.app/Contents/MacOS/Google Chrome"
)

# 불필요한 에러 메시지 삭제
options.add_experimental_option("excludeSwitches", ["enable-logging"])
# 최대화
options.add_argument('--start-maximized')

chrome_driver_path = (
    "/Users/powerjsv/Desktop/project/capstone/chromedriver-mac-arm64/chromedriver"
)

driver = webdriver.Chrome(chrome_driver_path, chrome_options=options)
driver.implicitly_wait(2)

/var/folders/9q/1dly5q8d5j9dmt8t1fwf05r80000gn/T/ipykernel_84406/2833979925.py:16: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_path, chrome_options=options)


## 광고 제거 함수

In [81]:
def remove_ad():
    try:
        iframe=driver.find_element(By.NAME,"aswift_3")
        driver.switch_to.frame(iframe)
        dismiss_button=driver.find_element(By.CSS_SELECTOR, "#dismiss-button")
        time.sleep(random.uniform(0,1))
        driver.execute_script("arguments[0].click();", dismiss_button)
        driver.switch_to.default_content()  # 메인프레임으로 이동
    except:
        print("Cannot find Ad")

In [82]:
# # 구,동, 카테고리 지역별로 반복문 돌리려 했던 코드.
# for place in places:
#     # driver.get(url+place)
#     driver.get(url + "화양동")

#     # 20개씩 5번 검색결과 더 보기 클릭
#     xpath_view_20_more = '//*[@id="map"]/button[2]/span'
#     for _ in range(4):
#         driver.find_element_by_xpath(xpath_view_20_more).click()
#         time.sleep(random.uniform(1,3))

#     # 각각의 식당 클릭
#     cnt=0
#     restaurant_find_selector = "div.RHeader > div > div.InfoHeader"

#     for i in driver.find_elements(By.CSS_SELECTOR, restaurant_find_selector):
#         cnt+=1
#         time.sleep(random.uniform(1, 3))
#         driver.execute_script("arguments[0].click();", i)

#         # 클릭시 리뷰가 새 창으로 뜨기 때문에 창을 이동해줌

#         tabs = driver.window_handles
#         driver.switch_to.window(tabs[1])
#         print("Move To New Tab")

#         # 리뷰 더보기 버튼 안 나올때까지 클릭
#         while True:
#             try:
#                 more_review_selector = "#div_more_review > button"
#                 more_review=driver.find_element(By.CSS_SELECTOR,more_review_selector)
#                 driver.execute_script("arguments[0].click();", more_review)
#                 time.sleep(random.uniform(1,3))
#             except:
#                 break
#         restaurant=[]
#         review=[]

#         # 창 닫고 원래 창으로 돌아옴
#         driver.close()
#         print("Tab closed")
#         tabs=driver.window_handles
#         driver.switch_to.window(tabs[0])
#         remove_ad()
#         if cnt == 3:
#             break
#     break

# 식당 정보 크롤링 함수

In [83]:
# div_profile > div.s-list.pic-grade > div.tit-point > h1

In [84]:
def crawl_restaurant_info():
    remove_ad()
    driver.switch_to.window(driver.window_handles[1])
    
    restaurant_table=[]
    rst_columns=['name','category','sub_category','address','contact','platform']

    restaurant_menu_table=[]
    rm_columns=['rst_name','menu_name','price','platform']

    opening_hours_table=[]
    oh_columns=['rst_name','weekday','hours','platform']

    rst_info=[]
    rst_menu_info=[]
    rst_oh_info=[]

    rst_name = driver.find_element(
        By.CSS_SELECTOR, "#div_profile > div.s-list.pic-grade > div.tit-point > h1"
    )

    rst_info.append(rst_name.text)

    """
    음식점 : 0
    카페 : 1
    술집 : 2

    크롤링 카테고리에 맞게 변수 바꿔야함
    """

    """-------------------------------------------------------식당 기본정보 크롤링-------------------------------------------------------------------"""
    rst_info.append(category)

    # 서브 카테고리들 리스트로 받음
    try:
        sub_categorys = driver.find_elements(By.CSS_SELECTOR, "#div_profile > div.s-list.pic-grade > div.btxt > a.btxt")
        categories=[]
        for each_category in sub_categorys:
            categories.append(each_category.text)

    except:
        categories=None

    # 레스토랑 정보에 서브 카테고리 추가
    rst_info.append(categories)

    # 레스토랑 주소 추가
    try:
        rst_loc=driver.find_elements(By.CSS_SELECTOR, "#div_profile > div.s-list.basic-info > ul > li.locat > a")
        rst_loc_num = driver.find_element(By.CSS_SELECTOR, "#div_profile > div.s-list.basic-info > ul > li.locat > span")
        rst_address=""
        for i in rst_loc:
            rst_address+=i.text
            rst_address+=' '

        rst_address+=rst_loc_num.text
        print(rst_address)

    except:
        rst_address=None

    rst_info.append(rst_address)

    # 레스토랑 전화번호
    try:
        rst_contact = driver.find_element(By.CSS_SELECTOR, "#div_profile > div.s-list.basic-info > ul > li.tel")
        print(rst_contact.text)
    except:
        rst_contact=None
    rst_info.append(rst_contact.text)

    # 크롤링 플랫폼 : 네이버 0, 다이닝코드 1
    rst_info.append(platform)

    # Data frame 화 -> csv
    restaurant_table.append(rst_info)
    rst_table_df=pd.DataFrame(restaurant_table,columns=rst_columns)
    if not os.path.exists("info_hwayang_cafe_diningcode.csv"):
        print("make new csv")
        rst_table_df.to_csv("info_hwayang_cafe_diningcode.csv",sep=",",encoding='utf-8-sig',mode='w')
    else:
        print("add csv")
        rst_table_df.to_csv("info_hwayang_cafe_diningcode.csv",sep=",",encoding="utf-8-sig",mode="a",header=False,)

    del [[rst_table_df]]

    """-------------------------------------------------------식당 메뉴판 크롤링-------------------------------------------------------------------"""

    # 메뉴 더보기 클릭!
    try:
        driver.find_element(By.CSS_SELECTOR, "#div_detail > div.menu-info.short > p.r > a > span").click()
    except:
        pass

    # 메뉴판 크롤링
    try:
        menus=driver.find_elements(By.CSS_SELECTOR,'#div_detail > div.menu-info > ul > li > p.l-txt.Restaurant_MenuItem > span.Restaurant_Menu')
        prices = driver.find_elements(By.CSS_SELECTOR,"#div_detail > div.menu-info > ul > li > p.r-txt.Restaurant_MenuPrice")
        for i in range(len(menus)):
            rst_menu_info=[]
            rst_menu_info.append(rst_name.text)
            rst_menu_info.append(menus[i].text)
            rst_menu_info.append(prices[i].text)
            rst_menu_info.append(platform)

            restaurant_menu_table.append(rst_menu_info)

        rst_menu_table_df=pd.DataFrame(restaurant_menu_table,columns=rm_columns)
        if not os.path.exists("menu_hwayang_cafe_diningcode.csv"):
            print("make new csv")
            rst_menu_table_df.to_csv("menu_hwayang_cafe_diningcode.csv",sep=",",encoding='utf-8-sig',mode='w')
        else:
            print("add csv")
            rst_menu_table_df.to_csv("menu_hwayang_cafe_diningcode.csv",sep=",",encoding="utf-8-sig",mode="a",header=False,)

        del [[rst_menu_table_df]]
    except:
        print("No menus")

    """-------------------------------------------------------식당 영업시간 크롤링-------------------------------------------------------------------"""
    # 영업시간 더보기 버튼 있으면 클릭
    try:
        driver.find_element(By.CSS_SELECTOR,"#div_detail > div.busi-hours.short > p > a > span").click()
    except:
        print("no more btn")

    try:
        # 오늘의 영업시간 크롤링
        today = driver.find_element(
            By.CSS_SELECTOR,
            "#div_detail > div.busi-hours-today > ul > li:nth-child(1) > p.l-txt > strong",
        )

        today_times = driver.find_element(
            By.CSS_SELECTOR,
            "#div_detail > div.busi-hours-today > ul > li:nth-child(1) > p.r-txt",
        )

        if today.text:
            rst_oh_info.append(rst_name.text)
            rst_oh_info.append(today.text)
            rst_oh_info.append(today_times.text)
            rst_oh_info.append(platform)
        opening_hours_table.append(rst_oh_info)

        # 일반적인 영업시간 크롤링
        weekdays = driver.find_elements(By.CSS_SELECTOR, "#div_detail > div.busi-hours > ul > li > p.l-txt")
        print("len =",len(weekdays))
        work_times = driver.find_elements(By.CSS_SELECTOR, "#div_detail > div.busi-hours > ul > li > p.r-txt")

        for i in range(len(weekdays)):
            rst_oh_info=[]
            if weekdays[i].text:
                rst_oh_info.append(rst_name.text)
                rst_oh_info.append(weekdays[i].text)
                rst_oh_info.append(work_times[i].text)
                rst_oh_info.append(platform)
            opening_hours_table.append(rst_oh_info)

        oh_df = pd.DataFrame(opening_hours_table, columns=oh_columns)
        if not os.path.exists("opening_hours_hwayang_cafe_diningcode.csv"):
            print("make new csv")
            oh_df.to_csv(
                "opening_hours_hwayang_cafe_diningcode.csv",
                sep=",",
                encoding="utf-8-sig",
                mode="w",
            )
        else:
            print("add csv")
            oh_df.to_csv(
                "opening_hours_hwayang_cafe_diningcode.csv",
                sep=",",
                encoding="utf-8-sig",
                mode="a",
                header=False,
            )

        del [[oh_df]]

    except:
        print("no opening hours")

# 식당 리뷰 및 태그 크롤링 함수

In [85]:
def crawl_restaurant_review():
    remove_ad()

    driver.switch_to.window(driver.window_handles[1])
    review_table=[]
    review_columns=['rst_name','user_name','review_text','u_rst_tag','ate_menus','date','platform']

    rst_name = driver.find_element(
        By.CSS_SELECTOR, "#div_profile > div.s-list.pic-grade > div.tit-point > h1"
    )

    review_blocks= driver.find_elements(
        By.CSS_SELECTOR, ".latter-graph"
    )

    for block in review_blocks:
        info=[]
        print(rst_name.text)
        info.append(rst_name.text)

        try:
            user_name=block.find_element(By.CSS_SELECTOR,"p.person-grade > span.btxt > strong")
            print(user_name.text)
            info.append(user_name.text)
        except:
            user_name=None
            info.append(None)    

        try:
            review_text = block.find_element(By.CSS_SELECTOR, "p.review_contents.btxt")
        except:
            review_text=None
        review_txt=review_text.text.replace("\n", " ")
        print(review_txt)
        info.append(review_txt)

        try:
            u_rst_tags=block.find_element(By.CSS_SELECTOR ,"div.keyword")
            info.append(u_rst_tags.text)
        except:
            info.append(None)

        # ate_menus=block.find_elements(By.CSS_SELECTOR, "p.tags > span.icon")
        # for each_menu in ate_menus:
        #     print(each_menu.text)

        try:
            ate_menus=block.find_element(By.CSS_SELECTOR ,"p.tags")
            info.append(ate_menus.text)
        except:
            info.append(None)

        try:
            date=block.find_element(By.CSS_SELECTOR, "div.date")
            info.append(date.text)
        except:
            info.append(None)

        platform=1
        info.append(platform)

        print("----------------"*15)
        review_table.append(info)

    review_table_df=pd.DataFrame(review_table,columns=review_columns)
    if not os.path.exists("review_hwayang_cafe_diningcode.csv"):
        print("make new csv")
        review_table_df.to_csv("review_hwayang_cafe_diningcode.csv",sep=",",encoding='utf-8-sig',mode='w')
    else:
        print("add csv")
        review_table_df.to_csv("review_hwayang_cafe_diningcode.csv", sep=",", encoding="utf-8-sig", mode="a",header=False)

    # 메모리관리를 위한 df 삭제
    del [[review_table_df]]

# 크롤링

In [86]:
# 러프하게 location 입력해서 돌리자
driver.get(url + location)

# 검색결과 더 보기 클릭
for i in range(4):
    try:
        driver.find_element(
            By.CSS_SELECTOR, "#map > button.SearchMore.upper > span"
        ).click()
        time.sleep(0.5)
    except:
        break
# 각각의 식당 클릭
# cnt = 0
restaurant_find_selector = "div.RHeader > div > div.InfoHeader"
restaurant_finder= driver.find_elements(By.CSS_SELECTOR, restaurant_find_selector)

#i=0부터 도는데 혹시 오류 뜨면 디버깅 후 오류난 인덱스부터 돌리면 됨
i=81
while i<len(restaurant_finder):

    for _ in range(4):
        try:
            driver.find_element(
                By.CSS_SELECTOR, "#map > button.SearchMore.upper > span"
            ).click()
            time.sleep(0.5)
        except:
            break
    time.sleep(2)
    restaurant_finder = driver.find_elements(By.CSS_SELECTOR, restaurant_find_selector)

    for _ in range(4):
        try:
            driver.find_element(
                By.CSS_SELECTOR, "#map > button.SearchMore.upper > span"
            ).click()
            time.sleep(0.5)
        except:
            break

    # driver.execute_script("arguments[0].scrollIntoView({block : 'center'});",i)
    driver.execute_script("arguments[0].click();", restaurant_finder[i])

    # 클릭시 리뷰가 새 창으로 뜨기 때문에 창을 이동해줌

    tabs = driver.window_handles
    driver.switch_to.window(tabs[1])
    print("Move To New Tab")
    # 크롤링 코드 시작
    print(f"{i}번째 rst")
    crawl_restaurant_info()
    # 리뷰 더보기 버튼 안 나올때까지 클릭
    while True:
        try:
            more_review_selector = "#div_more_review > button"
            more_review = driver.find_element(By.CSS_SELECTOR, more_review_selector)
            driver.execute_script("arguments[0].click();", more_review)
            time.sleep(0.5)
        except:
            break
    # 크롤링 코드 시작
    # 식당 리뷰 크롤링

    time.sleep(0.2)
    crawl_restaurant_review()
    # 크롤링 코드 끝
    # 창 닫고 원래 창으로 돌아옴
    driver.close()
    print("Tab closed")
    time.sleep(0.2)
    tabs = driver.window_handles
    driver.switch_to.window(tabs[0])
    remove_ad()
    i+=1
    


Move To New Tab
81번째 rst
Cannot find Ad
서울특별시 광진구 화양동 3-37 1층
0507-1329-1522
add csv
add csv
len = 12
add csv
Cannot find Ad
콩카페 서울건대점
euzz
🍀한줄평 : 코코넛커피스무디는 필수, 신규 매장으로 깔끔  콩카페 코코넛커피스무디를 드디어 먹어보네요. 화장실도 깔끔하고 면봉이 구비되어있어서 좋았습니다.
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
콩카페 서울건대점
루크
베트남 커피 처음 마셔보는데 맛있네요. 코코넛 연유커피는 약간 더위사냥 맛이나요
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
add csv
Tab closed
Move To New Tab
82번째 rst
Cannot find Ad
서울특별시 광진구 화양동 1
02-450-3977
add csv
add csv
len = 6
add csv
Cannot find Ad
메가커피 건국대점
livefortooday
가성비 너무 좋고 맛있어서 더 좋습니다!! 메가커피 자주 가는데 매장도 깔끔하고 좋아요!!
------------------------

In [87]:
i

100